In [1]:
import os
from transformers import pipeline
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_community.chat_models import ChatHuggingFace
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, HuggingFaceHubEmbeddings
from langchain_community.document_loaders import WebBaseLoader, UnstructuredURLLoader
from transformers import AutoModelForCausalLM, AutoTokenizer



In [2]:
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction/")
docs = loader.load()
len(docs)

1

In [3]:
embeddings = HuggingFaceEmbeddings()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)

/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

llm = HuggingFaceEndpoint(
    task="text-generation",
    repo_id="meta-llama/Meta-Llama-Guard-2-8B",
)
chat_model = ChatHuggingFace(llm=llm, token=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/felixachter/.cache/huggingface/token
Login successful


/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

messages = [
    # HumanMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]
chat_model.invoke(messages)

HfHubHTTPError:  (Request ID: 8_fz3x5646oMo1TkXadnB)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-Guard-2-8B.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model meta-llama/Meta-Llama-Guard-2-8B is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [8]:

model_name = "meta-llama/Meta-Llama-Guard-2-8B"
llm = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
retriever = vector.as_retriever()

/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to information relevant to the conversation.")
])
history_aware_chain = create_history_aware_retriever(llm, retriever, prompt)